In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd

from src.features import build_features
import src.features.pabellon as pb

pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

## Obtencion de casos por area de influencia INT

In [ ]:
# Cargar datos
casos_area_de_influencia = build_features.leer_casos_area_de_influencia(
    COLUMNAS_POBLACION_INTERES
)

## Obtencion de porcentajes de quirurgicos

In [14]:
# Define la ruta donde estan los porcentajes de quirurgicos
RUTA_PLANILLA_PORCENTAJES = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_INT.xlsx"
)

# Lee porcentajes y especialidades quirurgicas de trazadoras
porcentajes_quirurgicos = pd.read_excel(
    RUTA_PLANILLA_PORCENTAJES,
    sheet_name="porcentaje_trazadoras",
    usecols=["Diagnostico", "Porcentaje Pacientes Quirúrgicos", "Especialidad Quirúrgica"],
)

# Deja los diagnosticos que SI aportan a los quirofanos (porcentaje > 0)
porcentajes_quirurgicos = porcentajes_quirurgicos.query("`Porcentaje Pacientes Quirúrgicos` > 0")

# Formatea la columna de diagnosticos
porcentajes_quirurgicos["Diagnostico"] = (
    porcentajes_quirurgicos["Diagnostico"].str.split(" - ").str[0]
)

# Deja los diagnosticos como el indice
porcentajes_quirurgicos = porcentajes_quirurgicos.set_index("Diagnostico")

Una vez se han obtenido los casos de area de influencia totales, y tambien los porcentajes de quirurgicos, se procederá a solamente dejar la información de los diagnósticos que SÍ aportan a los quirofanos (un porcentaje > 0).

Posteriormente, se reasignarán los diagnósticos para agrupar algunas patologías.

In [16]:
# Filtra los casos de area de influencia solamente a los diagnosticos que tengan un porcentaje de
# quirofanos mayor a 0
casos_area_de_influencia_para_quir = casos_area_de_influencia[
    casos_area_de_influencia.index.isin(porcentajes_quirurgicos.index)
]

Luego, se obtendrán todos los diagnósticos que deben ser buscados en la base de datos de Pabellon y GRD del INT.

In [18]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_area_de_influencia_para_quir.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_area_de_influencia_para_quir.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_quir = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtencion de tiempos quirurgicos entre 2015 y 2022 en pabellon para diagnosticos relevantes

Estos se obtendran a partir de la union de la base de GRD (2015 a 2022, tiene los diagnosticos codificados) y Pabellon (2015 a 2022, tiene los tiempos quirurgicos).

In [39]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_duracion_int_q/grd_interno_y_pabellon.csv")
grd_y_pabellon["duracion"] = pd.to_timedelta(grd_y_pabellon["duracion"])

In [40]:
# Filtra solamente los diagnosticos que aportan a quirurgico
grd_y_pabellon_relevantes = grd_y_pabellon.query(
    "diag_01_principal_cod.isin(@diagnosticos_a_buscar_en_quir)"
).copy()

## Reasginar diagnosticos

In [41]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    diagnosticos_cambiados = grd_y_pabellon_relevantes["diag_01_principal_cod"].replace(
        diagnosticos_antiguos, diagnostico_nuevo
    )
    grd_y_pabellon_relevantes["diag_01_principal_cod"] = diagnosticos_cambiados

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


## Obtener tiempos quirurgicos

In [42]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
resumen_duraciones_diags_relevantes = grd_y_pabellon_relevantes.groupby(["diag_01_principal_cod"])[
    "duracion"
].describe()["75%"]

In [ ]:
# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos, resumen_duraciones_diags_relevantes
)

# Calcular horas laborales
horas_laborales = pb.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

# Obtiene la suma total de pabellones
suma_total_pabellones = cantidad_de_pabellones_necesarios.sum()

Tiempo utilizado en pabellón calculado (en horas):
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      |        2017        |        2018        |        2019        |        2020        |        2021        |        2022        |        2023        |        2024        |        2025        |        2026        |        2027        |        2028        |        2029        |        2030        |        2031        |        2032        |        2033        |        2034        |        2035        |
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

## Complicaciones

In [ ]:
# Define las complicaciones a buscar
a_buscar_operaciones = {
    "ecmo": r"ECMO",
    "trasplantes": r"TRASPLANTE|TRANSPLANTE",
    "aseos": r"ASEO",
    "drenajes": r"DRENAJE",
    "traqueostomias": r"TRAQUEOSTOMIA|TRAQUEOSTOMÍA",
    "reintervenciones_reoperaciones": r"REINTERVENCION|REOPERACION|REINTERVENCIÓN|REOPERACIÓN",
}

# Define los diagnosticos que son complicaciones
a_buscar_diagnosticos = {
    "empiemas": r"EMPIEMA",
    "rupturas": r"ANEURISMA",
}

# Busca los nombres de las operaciones en la base de pabellon
resultados_operaciones = pb.iterar_en_complicaciones_a_buscar(
    grd_y_pabellon, a_buscar_operaciones, "intervencion_quirurgica"
)

# Busca los diagnosticos en el primer diagnostico y segundo
resultados_diagnosticos = pb.iterar_en_complicaciones_a_buscar(
    grd_y_pabellon, a_buscar_diagnosticos, "diagnostico"
)

# Obtiene resumen de complicaciones
resumen_complicaciones = pd.concat([resultados_operaciones, resultados_diagnosticos])

In [ ]:
# Une los datos de complicaciones y los casos de especialidad
casos_complicaciones = (
    casos_especialidad.merge(
        resumen_complicaciones[["fraccion", "tiempo_operacion_75%", "complicacion"]],
        how="left",
        left_index=True,
        right_index=True,
    )
    .reset_index()
    .sort_values(["complicacion", "especialidades"])
    .set_index(["complicacion", "especialidades"])
)

# Obtiene los casos de complicaciones
casos_complicaciones[COLUMNAS_POBLACION_INE] = casos_complicaciones[COLUMNAS_POBLACION_INE].mul(
    casos_complicaciones["fraccion"], axis=0
)

# Obtiene el tiempo de pabellon por las complicaciones
tiempo_utilizado_pabellon_complicaciones_horas = (
    casos_complicaciones[COLUMNAS_POBLACION_INE]
    .mul(casos_complicaciones["tiempo_operacion_75%"], axis=0)
    .apply(lambda x: x.dt.total_seconds() / 3600)
)

# Divide por las horas laborales
pabellones_por_complicaciones = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_complicaciones_horas, horas_laborales
)

# # Obtiene los pabellones de emergencia por anio
suma_total_pabellones_complicaciones = pabellones_por_complicaciones.sum()

Cantidad de pabellones necesarios calculada:
+--------------------+----------------------+----------------------+----------------------+---------------------+----------------------+---------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+
|                    |         2017         |         2018         |         2019         |        2020         |         2021         |        2022         |         2023         |         2024         |         2025         |         2026         |         2027         |         2028         |         2029         |         2030         |         2031         |         2032         |         2033         |         2034         |         2035         |
+--------------------+-----------------

In [ ]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = (
    pd.read_excel(
        "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
    )
    .drop(columns="Porcentaje Hospitalizados de Ambulatorios")
    .sort_values("Diagnostico")
)
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2035
resumen_quirurgicos = pd.DataFrame(
    {
        "porcentaje_quirurgico": porcentajes_de_quirurgicos,
        "casos_quirurgicos_2035": casos_quirurgicos["2035"],
        "tiempo_quirurgico_75%_2019_a_2022": resumen_duraciones_diags_relevantes,
        "horas_pabellon_2035": tiempo_utilizado_pabellon_horas["2035"],
        "horas_laborales_2035_pabellon_12_hrs": horas_laborales["2035"],
        "cantidad_de_pabellones": cantidad_de_pabellones_necesarios["2035"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="left", left_index=True, right_index=True
)

In [ ]:
print(f"> Pabellones 2035: {resumen_minsal['cantidad_de_pabellones'].sum():.2f}")
print(f"> Pabellones de Emergencia 2035: {suma_total_pabellones_complicaciones['2035']:.2f}")

> Pabellones 2035: 5.44
> Pabellones de Emergencia 2035: 0.97


In [ ]:
a_guardar = {
    "casos_area_de_influencia": casos_area_de_influencia,
    "porcentajes_quirurgicos": porcentajes_de_quirurgicos,
    "casos_quirurgicos": casos_quirurgicos,
    "casos_por_especialidad": casos_especialidad,
    "casos_complicaciones": casos_complicaciones,
    "resumen_duraciones_int_q_rel": resumen_duraciones_diags_relevantes,
    "tiempo_utilizado_pabellon": tiempo_utilizado_pabellon_horas,
    "horas_laborales_por_anio": horas_laborales,
    "pabellones_desg": cantidad_de_pabellones_necesarios,
    "pabellones": suma_total_pabellones,
    "pabellones_desg_emergencia": pabellones_por_complicaciones,
    "pabellones_emergencia": suma_total_pabellones_complicaciones,
    "resumen_MINSAL": resumen_minsal,
}

In [ ]:
with pd.ExcelWriter("../data/interim/estimacion_pabellones_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)